## Project


## Importing the Packages

In [8]:
import numpy as np

In [9]:
np.set_printoptions(suppress=True, linewidth=100, precision=2)

## Importing the Data

- np.loadtxt() didn't work -> use np.genfromtxt()
- first row of data set in non-numeric. Use skip_header=1 to omit it.
- Also set autostrip=True to remove whitespace; prevents distortion of columns

In [10]:
raw_data_np = np.genfromtxt("loan-data.csv", delimiter=";", skip_header=1, autostrip=True)
raw_data_np

array([[48010226.  ,         nan,    35000.  , ...,         nan,         nan,     9452.96],
       [57693261.  ,         nan,    30000.  , ...,         nan,         nan,     4679.7 ],
       [59432726.  ,         nan,    15000.  , ...,         nan,         nan,     1969.83],
       ...,
       [50415990.  ,         nan,    10000.  , ...,         nan,         nan,     2185.64],
       [46154151.  ,         nan,         nan, ...,         nan,         nan,     3199.4 ],
       [66055249.  ,         nan,    10000.  , ...,         nan,         nan,      301.9 ]])

## Checking for Incomplete Data

- check for all missing values in the dataset
    - call np.isnan(raw_data_np) and chain .sum()

In [11]:
np.isnan(raw_data_np).sum()

88005

- fill out missing values with np.nanmax(raw_data_np) + 1 
    - we want our missing values to be replaced with values that aren't part of the dataset 
        - serves as a filler for missing entries

- create temporary_mean with np.nanmean(raw_data_np, axis=0)
    - this will hold the mean for every column

In [12]:
temporary_fill = np.nanmax(raw_data_np) + 1
temporary_mean = np.nanmean(raw_data_np, axis=0)

C:\Users\nickpetras\AppData\Local\Temp\ipykernel_17460\2996838478.py:2: RuntimeWarning: Mean of empty slice
  temporary_mean = np.nanmean(raw_data_np, axis=0)


- based on warningerror above, we have at least one empty column OR a column(s) of only strings

- to check how many columns consist of entirely text values, display temporary_mean (will return NaNs for text values)
    - if a single row isn't NaN, the nanmean will return its value as the mean
        - any column with a mean of NaN contains no numbers

In [13]:
temporary_mean

array([54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
            440.92,         nan,         nan,         nan,         nan,         nan,     3143.85])

- should split data into smaller arrays
    - 1 containing numeric values
    - another containing strings

In [14]:
temporary_stats = np.array([np.nanmin(raw_data_np, axis=0),
                            temporary_mean,
                            np.nanmax(raw_data_np, axis=0)])

C:\Users\nickpetras\AppData\Local\Temp\ipykernel_17460\902197374.py:1: RuntimeWarning: All-NaN slice encountered
  temporary_stats = np.array([np.nanmin(raw_data_np, axis=0),
C:\Users\nickpetras\AppData\Local\Temp\ipykernel_17460\902197374.py:3: RuntimeWarning: All-NaN slice encountered
  np.nanmax(raw_data_np, axis=0)])


- warning indicates we're plugging in empty slices
    - these are full of text -> no need to worry about them now

In [15]:
temporary_stats

array([[  373332.  ,         nan,     1000.  ,         nan,     1000.  ,         nan,        6.  ,
              31.42,         nan,         nan,         nan,         nan,         nan,        0.  ],
       [54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
             440.92,         nan,         nan,         nan,         nan,         nan,     3143.85],
       [68616519.  ,         nan,    35000.  ,         nan,    35000.  ,         nan,       28.99,
            1372.97,         nan,         nan,         nan,         nan,         nan,    41913.62]])

- output consists of 2d array that holds three 1d arrays stacked ontop of each other
    - 1 = nanmin of raw_data_np
    - 2 = temporary_mean of raw_data_np
    - 3 = nanmax of raw_data_np

## Splitting the Dataset

### Splitting the Columns
- splitting into numerical and string data

- we used `temporary_mean = np.nanmean(raw_data_np, axis=0)` to figure out which columns stored text/numeric data 

In [16]:
column_strings = np.argwhere(np.isnan(temporary_mean)).squeeze() # use .squeeze() to turn into 1d array
column_strings

array([ 1,  3,  5,  8,  9, 10, 11, 12], dtype=int64)

- default argument for `np.argwhere()` tests wherher the values are different from 0
- if columns contain only text, mean == NaN -> returns True. True != 0, thus `np.argwhere()` returns the index of this column in the original dataset.

In [17]:
column_numbers = np.argwhere(np.isnan(temporary_mean) == False).squeeze()
column_numbers

array([ 0,  2,  4,  6,  7, 13], dtype=int64)

- column_numbers contains indices of where numeric values are located
- can now use the usecols= argument to split our columns accordingly when reimporting the original dataset as two separate arrays

### Re-importing the Dataset

In [18]:
loan_data_strings = np.genfromtxt("loan-data.csv", delimiter=";", skip_header=1, autostrip=True, usecols=column_strings, dtype=str)

loan_data_strings

array([['May-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

In [19]:
loan_data_numeric = np.genfromtxt("loan-data.csv", delimiter=";", skip_header=1, autostrip=True, usecols=column_numbers, filling_values=temporary_fill)

loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]])

### The Names of the Columns

- `skip_footer=raw_data_np.shape[0]`
    - this means we're skipping all rows except the header (index position 0)

In [20]:
header_full = np.genfromtxt("loan-data.csv", delimiter=";", skip_footer=raw_data_np.shape[0], autostrip=True, dtype=str)

header_full

array(['id', 'issue_d', 'loan_amnt', 'loan_status', 'funded_amnt', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state',
       'total_pymnt'], dtype='<U19')

- assign the header_full variable, specifying indexes of column_strings and column_numbers and assign these to two new variables of header_string and header_numeric.
    - spltting names of columns between the dtype of string and numbers

In [21]:
header_strings, header_numeric = header_full[column_strings], header_full[column_numbers]

In [22]:
header_strings

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [23]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

## Creating Checkpoints:

- checkpoints
    - places throughout our code where we store a copy of our dataset/parts of it
    - avoid losing progress if we overwrite variables
        - reliable practice when we need to clean/preprocess parts of a dataset 
        - failsafe

In [24]:
def checkpoint(file_name, checkpoint_header, checkpoint_data):
    np.savez(file_name, header = checkpoint_header, data = checkpoint_data)
    checkpoint_variable = np.load(file_name + ".npz")
    return(checkpoint_variable)

Function:
- 3 parameters = file_name, checkpoint_header, checkpoint_data
- creates a .npz file with a name we have assigned
- header= and data= -> proper names for the .npy(s) 
- checkpoint_variable -> loads the .npz created (specify .npz as np.load() can open .npy(s))
- checkpoint_variable is also the return value of the function -> loads the .npz


In [25]:
checkpoint_test = checkpoint("checkpoint-test", header_strings, loan_data_strings)

In [26]:
checkpoint_test["header"]

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [27]:
checkpoint_test["data"]

array([['May-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

use np.equal_array() to check if checkpoint function works as expected

In [28]:
np.array_equal(checkpoint_test["header"], header_strings)

True

In [29]:
np.array_equal(checkpoint_test["data"], loan_data_strings)

True

## Manipulating String Columns

In [30]:
header_strings

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

Manipulate issue_d to issue_date

In [31]:
header_strings[0] = "issue_date"

In [32]:
header_strings

array(['issue_date', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

### Issue Date

remove -15 from all rows in column index position 0

In [33]:
np.unique(loan_data_strings[:,0])

array(['', 'Apr-15', 'Aug-15', 'Dec-15', 'Feb-15', 'Jan-15', 'Jul-15', 'Jun-15', 'Mar-15',
       'May-15', 'Nov-15', 'Oct-15', 'Sep-15'], dtype='<U69')

In [34]:
loan_data_strings[:,0] = np.chararray.strip(loan_data_strings[:,0], "-15")

- represent months as integer values

In [35]:
months = np.array(['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

- months variable to store loan_data_strings[:,0] in order 
- null data will be assigned 0

In [36]:
for i in range(13): # 0->12 
        loan_data_strings[:,0] = np.where(loan_data_strings[:,0] == months[i],
                                          i,
                                          loan_data_strings[:,0])

- loop will assign the issue_date column the output of np.where()
- np.where() checks whether the value from the column equals a specific value from the months array previously defined
    - if so, it'll be assigned the corresponding numerical value
    - if not, the issue_date remains unchanged

- all missing values will be filled with 0

In [37]:
np.unique(loan_data_strings[:,0])

array(['0', '1', '10', '11', '12', '2', '3', '4', '5', '6', '7', '8', '9'], dtype='<U69')

### Loan Status

In [38]:
header_strings

array(['issue_date', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [39]:
np.unique(loan_data_strings[:,1])

array(['', 'Charged Off', 'Current', 'Default', 'Fully Paid', 'In Grace Period', 'Issued',
       'Late (16-30 days)', 'Late (31-120 days)'], dtype='<U69')

In [40]:
np.unique(loan_data_strings[:,1]).size

9

- change loan status to show whether the applicant is in a good or bad economic state -> 0s and 1s?
- lack of data == bad sign
- 'Late (16-30 days)' - indicates applicants are paying back once they get paid 
- 'Late (31-120 days)' - indicates applicants will probably default 
- 0 = bad
- 1 = good

In [41]:
status_bad = np.array(["", "Charged Off", "Default", "Late (31-120 days)"])

In [42]:
loan_data_strings[:,1] = np.where(np.isin(loan_data_strings[:,1], status_bad),0,1)

- no need for a for loop above
- can chain on np.isin within np.where
- if status_bad == True, assign loan_data_strings[:,1] values to 0, if not assign values to 1 
    - accomodates the both bits of logic

In [43]:
np.unique(loan_data_strings[:,1])

array(['0', '1'], dtype='<U69')

- ^ checks if the overwriting of values has worked

### Term

In [44]:
np.unique(loan_data_strings[:,2])

array(['', '36 months', '60 months'], dtype='<U69')

In [45]:
loan_data_strings[:,2] = np.chararray.strip(loan_data_strings[:,2], " months")
loan_data_strings[:,2]

array(['36', '36', '36', ..., '36', '36', '36'], dtype='<U69')

- rename term to term_months

In [46]:
header_strings[2] = "term_months"

In [47]:
header_strings

array(['issue_date', 'loan_status', 'term_months', 'grade', 'sub_grade', 'verification_status',
       'url', 'addr_state'], dtype='<U19')

- assign all missing values in the term_months column to 60
    - when we have missing values in CRM (credit risk modelling), we assume the worst.
    - 60 months (5yrs) is a long period to pay off a loan
        - the loan is difficult to pay off

In [48]:
loan_data_strings[:,2] = np.where(loan_data_strings[:,2] == '', 60, loan_data_strings[:,2])

- 36 months = good scenario
- 60 months = bad scenario

In [49]:
np.unique(loan_data_strings[:,2])

array(['36', '60'], dtype='<U69')

- two unique values suggests we can use 0s and 1s.
    - dependent on whether 36 and 60 are significant values    
        - in this case, they are

### Grade and Subgrade

In [50]:
header_strings

array(['issue_date', 'loan_status', 'term_months', 'grade', 'sub_grade', 'verification_status',
       'url', 'addr_state'], dtype='<U19')

In [51]:
np.unique(loan_data_strings[:,3])

array(['', 'A', 'B', 'C', 'D', 'E', 'F', 'G'], dtype='<U69')

In [52]:
np.unique(loan_data_strings[:,4])

array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
       'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
       'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69')

- if the data is complete, any information we can get from the grade column can also be obtained from the subgrade column

#### Filling Sub Grade

In [53]:
for i in np.unique(loan_data_strings[:,3])[1:]:
    loan_data_strings[:,4] = np.where((loan_data_strings[:,4] == '') & (loan_data_strings[:,3] == i),
                                      i + '5',
                                      loan_data_strings[:,4])

- if the sub-grade for an account is missing, but the grade is B (e.g.), it is more appropriate to assign that subgrade a value of B5

- `for i in np.unique(loan_data_strings[:,3])[1:]:` 
    - goes through all the unique grades after the 1st one which is empty space
- `loan_data_strings[:,4] = np.where((loan_data_strings[:,4] == '') & (loan_data_strings[:,3] == i),`
    - then for every grade, we go through the column and check is the subgrade is missing
        - checking if values from the fourth column (index position) are empty and is equal to the grade for the current iteration
            - if so, add 5(lowest subgrade) to the subgrade `i + '5'`
            - if not, leave it as is `loan_data_string[:,4]`

In [54]:
np.unique(loan_data_strings[:,4])

array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
       'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
       'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69')

- we still have missing data
    - some rows do not have either subgrade/grade

- run np.unique() and implement the return_counts= parameter with an argument of True to find out how many missing rows we have

In [55]:
np.unique(loan_data_strings[:,4], return_counts=True)

(array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
        'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
        'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69'),
 array([  9, 285, 278, 239, 323, 592, 509, 517, 530, 553, 633, 629, 567, 586, 564, 577, 391, 267,
        250, 255, 288, 235, 162, 171, 139, 160,  94,  52,  34,  43,  24,  19,  10,   3,   7,   5],
       dtype=int64))

- create a new subgrade to account for the missing values using np.where()
    - follow the grading convention (H1)
    - following CRM standards, anything with missing data = redundant 

In [56]:
for i in np.unique(loan_data_strings[:,3])[1:]:
    loan_data_strings[:,4] = np.where(loan_data_strings[:,4] == '',
                                      "H1",
                                      loan_data_strings[:,4])

In [57]:
np.unique(loan_data_strings[:,4])

array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5',
       'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5',
       'G1', 'G2', 'G3', 'G4', 'G5', 'H1'], dtype='<U69')

- information contained in grade is carried by subgrade
    - grade is no longer needed

#### Removing Grade

- delete the grade column 

In [58]:
loan_data_strings = np.delete(loan_data_strings, 3, axis=1)

In [59]:
loan_data_strings[:,3]

array(['C3', 'A5', 'B5', ..., 'A5', 'D2', 'A4'], dtype='<U69')

- remove grade from our header

In [60]:
header_strings = np.delete(header_strings, 3, axis=0)

In [61]:
header_strings

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

#### Converting Sub Grade

In [62]:
np.unique(loan_data_strings[:,3])

array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5',
       'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5',
       'G1', 'G2', 'G3', 'G4', 'G5', 'H1'], dtype='<U69')

- create a dictionary 
    - every key will be a unique grade represented as a string
    - associated values will designate a rank of trustworthiness (number)
        - A1 = highest
        - 36 values total

In [63]:
keys = list(np.unique(loan_data_strings[:,3]))                         
values = list(range(1, np.unique(loan_data_strings[:,3]).shape[0] + 1)) 
dict_sub_grade = dict(zip(keys, values))

In [64]:
dict_sub_grade

{'A1': 1,
 'A2': 2,
 'A3': 3,
 'A4': 4,
 'A5': 5,
 'B1': 6,
 'B2': 7,
 'B3': 8,
 'B4': 9,
 'B5': 10,
 'C1': 11,
 'C2': 12,
 'C3': 13,
 'C4': 14,
 'C5': 15,
 'D1': 16,
 'D2': 17,
 'D3': 18,
 'D4': 19,
 'D5': 20,
 'E1': 21,
 'E2': 22,
 'E3': 23,
 'E4': 24,
 'E5': 25,
 'F1': 26,
 'F2': 27,
 'F3': 28,
 'F4': 29,
 'F5': 30,
 'G1': 31,
 'G2': 32,
 'G3': 33,
 'G4': 34,
 'G5': 35,
 'H1': 36}

- use a for loop to iterate through all the unique subgrades and substitute each one with its associated numeric value

In [65]:
for i in np.unique(loan_data_strings[:,3]):
        loan_data_strings[:,3] = np.where(loan_data_strings[:,3] == i, 
                                          dict_sub_grade[i],
                                          loan_data_strings[:,3])

- `for i in np.unique(loan_data_strings[:,3]):`
    - for loop goes through the subgrade array
- `loan_data_strings[:,3] = np.where(loan_data_strings[:,3] == i, `
    - using np.where() to find values equal to the subgrade we're iterating over
    - if condition is satisfied, use the subgrade as a key to the dictionary to pass the numeric value associated with that subgrade
    - if not, don't alter the initial subgrade

In [66]:
np.unique(loan_data_strings[:,3])

array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36',
       '4', '5', '6', '7', '8', '9'], dtype='<U69')

### Verification Status

In [67]:
header_strings

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [68]:
np.unique(loan_data_strings[:,4])

array(['', 'Not Verified', 'Source Verified', 'Verified'], dtype='<U69')

- empty space equivalent to not verified -> combine
- source verified == loan applications which include investor backing
- verified can be combined with source verified

- bad = 0
- good = 1

In [69]:
loan_data_strings[:,4] = np.where((loan_data_strings[:,4] == "") | (loan_data_strings[:,4] == "Not Verified"), 0, 1)

In [70]:
np.unique(loan_data_strings[:,4])

array(['0', '1'], dtype='<U69')

### URL

- remove the url link but keep the unique loan_id values

In [71]:
loan_data_strings[:,5]

array(['https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', ...,
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249'], dtype='<U69')

In [72]:
np.chararray.strip(loan_data_strings[:,5], "https://www.lendingclub.com/browse/loanDetail.action?loan_id=")

chararray(['48010226', '57693261', '59432726', ..., '50415990', '46154151', '66055249'],
          dtype='<U69')

In [73]:
loan_data_strings[:,5] = np.chararray.strip(loan_data_strings[:,5], "https://www.lendingclub.com/browse/loanDetail.action?loan_id=")

In [74]:
header_full

array(['id', 'issue_d', 'loan_amnt', 'loan_status', 'funded_amnt', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state',
       'total_pymnt'], dtype='<U19')

In [75]:
loan_data_numeric[:,0].astype(dtype=np.int32)

array([48010226, 57693261, 59432726, ..., 50415990, 46154151, 66055249])

In [76]:
loan_data_strings[:,5].astype(dtype=np.int32)

array([48010226, 57693261, 59432726, ..., 50415990, 46154151, 66055249])

In [77]:
np.array_equal(loan_data_numeric[:,0].astype(dtype=np.int32), loan_data_strings[:,5].astype(dtype=np.int32))

True

- get rid of URL column

In [78]:
loan_data_strings = np.delete(loan_data_strings, 5, axis=1)
header_strings = np.delete(header_strings, 5)

In [79]:
loan_data_strings[:,5]

array(['CA', 'NY', 'PA', ..., 'CA', 'OH', 'IL'], dtype='<U69')

In [80]:
header_strings

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'addr_state'], dtype='<U19')

In [81]:
loan_data_numeric[:,0]

array([48010226., 57693261., 59432726., ..., 50415990., 46154151., 66055249.])

In [82]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

### State Address

In [83]:
header_strings

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'addr_state'], dtype='<U19')

In [84]:
header_strings[5] = "state_address"

- create a tuple of states_names and states_counts equal to `np.unique(loan_data_strings[:,5], return_counts=True)`
- create states_count_sorted and assign it to an argsort of the states_count in descending order
- call states_names and states_counts with states_count_sorted as the index for both variables

- states will be arranged according to the number of applicants from each one

In [85]:
states_names, states_counts = np.unique(loan_data_strings[:,5], return_counts=True)
states_count_sorted = np.argsort(-states_counts)
states_names[states_count_sorted], states_counts[states_count_sorted]

(array(['CA', 'NY', 'TX', 'FL', '', 'IL', 'NJ', 'GA', 'PA', 'OH', 'MI', 'NC', 'VA', 'MD', 'AZ',
        'WA', 'MA', 'CO', 'MO', 'MN', 'IN', 'WI', 'CT', 'TN', 'NV', 'AL', 'LA', 'OR', 'SC', 'KY',
        'KS', 'OK', 'UT', 'AR', 'MS', 'NH', 'NM', 'WV', 'HI', 'RI', 'MT', 'DE', 'DC', 'WY', 'AK',
        'NE', 'SD', 'VT', 'ND', 'ME'], dtype='<U69'),
 array([1336,  777,  758,  690,  500,  389,  341,  321,  320,  312,  267,  261,  242,  222,  220,
         216,  210,  201,  160,  156,  152,  148,  143,  143,  130,  119,  116,  108,  107,   84,
          84,   83,   74,   74,   61,   58,   57,   49,   44,   40,   28,   27,   27,   27,   26,
          25,   24,   17,   16,   10], dtype=int64))

- Missing IA (Iowa)
    - left as a baseline benchmark 

- when doing research/analysis on a variable with many categories, it is normal to pick one as a benchmark and include dummy variables for the rest
- one without a dummy variable will serve as the base case and we will either increase/decrease the rest based on their coefficients - in our case this will be Iowa

- there are more applications with missing/unreported addresses than there are for 45 of the other states ( refer to '' )
    - not enough data to examine each one individually

- can assign a unique value to each state (as we did with subgrades) 
    - this will allow outliers to have a big influence on the coefficients for less represented states 
        - the more categories a variable has, the fewer data will be available for each one
            - states with fewer applications will be more vulnerable to have their coefficients affected by outliers
                - to solve this problem, we need to group these states according to a common characteristic - THEIR REGION
                    - need to resolve missing values

In [86]:
loan_data_strings[:,5] = np.where(loan_data_strings[:,5] == "", 0, loan_data_strings[:,5])

In [87]:
states_west = np.array(['WA', 'OR','CA','NV','ID','MT', 'WY','UT','CO', 'AZ','NM','HI','AK'])
states_south = np.array(['TX','OK','AR','LA','MS','AL','TN','KY','FL','GA','SC','NC','VA','WV','MD','DE','DC'])
states_midwest = np.array(['ND','SD','NE','KS','MN','IA','MO','WI','IL','IN','MI','OH'])
states_east = np.array(['PA','NY','NJ','CT','MA','VT','NH','ME','RI'])

https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf

In [88]:
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_west), 1, loan_data_strings[:,5])
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_south), 2, loan_data_strings[:,5])
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_midwest), 3, loan_data_strings[:,5])
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_east), 4, loan_data_strings[:,5])

In [89]:
np.unique(loan_data_strings[:,5])

array(['0', '1', '2', '3', '4'], dtype='<U69')

## Converting to Numbers

In [90]:
loan_data_strings

array([['5', '1', '36', '13', '1', '1'],
       ['0', '1', '36', '5', '1', '4'],
       ['9', '1', '36', '10', '1', '4'],
       ...,
       ['6', '1', '36', '5', '1', '1'],
       ['4', '1', '36', '17', '1', '3'],
       ['12', '1', '36', '4', '0', '3']], dtype='<U69')

In [91]:
loan_data_strings = loan_data_strings.astype(int)

In [92]:
loan_data_strings

array([[ 5,  1, 36, 13,  1,  1],
       [ 0,  1, 36,  5,  1,  4],
       [ 9,  1, 36, 10,  1,  4],
       ...,
       [ 6,  1, 36,  5,  1,  1],
       [ 4,  1, 36, 17,  1,  3],
       [12,  1, 36,  4,  0,  3]])

### Checkpoint 1: Strings

In [93]:
checkpoint_strings = checkpoint("Checkpoint-Strings", header_strings, loan_data_strings)

In [94]:
checkpoint_strings["header"]

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'state_address'], dtype='<U19')

In [95]:
checkpoint_strings["data"]

array([[ 5,  1, 36, 13,  1,  1],
       [ 0,  1, 36,  5,  1,  4],
       [ 9,  1, 36, 10,  1,  4],
       ...,
       [ 6,  1, 36,  5,  1,  1],
       [ 4,  1, 36, 17,  1,  3],
       [12,  1, 36,  4,  0,  3]])

In [96]:
np.array_equal(checkpoint_strings["data"], loan_data_strings)

True

## Manipulating Numeric Columns

In [97]:
loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]])

In [98]:
np.isnan(loan_data_numeric).sum()

0

- all missing values have been set to `temporary_fill`

- substitute fillers with worst possible values

### Substitute "Filler" Values

In [99]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

#### ID

- check all rows in the first column 

In [100]:
np.isin(loan_data_numeric[:,0], temporary_fill).sum()

0

In [101]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

#### Temporary Stats

In [102]:
temporary_stats[:, column_numbers]

array([[  373332.  ,     1000.  ,     1000.  ,        6.  ,       31.42,        0.  ],
       [54015809.19,    15273.46,    15311.04,       16.62,      440.92,     3143.85],
       [68616519.  ,    35000.  ,    35000.  ,       28.99,     1372.97,    41913.62]])

In [103]:
temporary_stats

array([[  373332.  ,         nan,     1000.  ,         nan,     1000.  ,         nan,        6.  ,
              31.42,         nan,         nan,         nan,         nan,         nan,        0.  ],
       [54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
             440.92,         nan,         nan,         nan,         nan,         nan,     3143.85],
       [68616519.  ,         nan,    35000.  ,         nan,    35000.  ,         nan,       28.99,
            1372.97,         nan,         nan,         nan,         nan,         nan,    41913.62]])

- above is:
    - temporary min
    - temporary mean
    - temporary max
        - for our header_numeric 

#### Funded Amount

In [104]:
loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]])

In [105]:
loan_data_numeric[:,2] = np.where(loan_data_numeric[:,2] == temporary_fill, 
                                  temporary_stats[0, column_numbers[2]],
                                  loan_data_numeric[:,2])
loan_data_numeric[:,2]

array([35000., 30000., 15000., ..., 10000., 10000., 10000.])

In [106]:
temporary_stats[0,column_numbers[3]]

6.0

#### Loaned Amount, Interest Rate, Total Payment, Installment

In [107]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

In [108]:
for i in [1,3,4,5]:
    loan_data_numeric[:,i] = np.where(loan_data_numeric[:,i] == temporary_fill, temporary_stats[2, column_numbers[i]], loan_data_numeric[:,i])

In [109]:
loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  ,       28.99,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  ,       28.99,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  ,       28.99,     1372.97,     2185.64],
       [46154151.  ,    35000.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  ,       28.99,      309.97,      301.9 ]])

### Currency Change

#### The Exchange Rate

- use EUR-USD.csv to convert usd to euro 

In [110]:
EUR_USD = np.genfromtxt("EUR-USD.csv", delimiter=",", autostrip=True, skip_header=1, usecols=3)
EUR_USD

array([1.13, 1.12, 1.08, 1.11, 1.1 , 1.12, 1.09, 1.13, 1.13, 1.1 , 1.06, 1.09])

- focus on closing price 

In [111]:
loan_data_strings[:,0]

array([ 5,  0,  9, ...,  6,  4, 12])

- ^ values represents which month the loan was issued 
    - 0 = all dates the accounts were date wasn't provided

In [118]:
exchange_rate = loan_data_strings[:,0]

for i in range(1,13):
    exchange_rate = np.where(exchange_rate == i,
                             EUR_USD[i-1],
                             exchange_rate)
    
exchange_rate = np.where(exchange_rate == 0, np.mean(EUR_USD), exchange_rate)


exchange_rate


array([1.1 , 1.11, 1.13, ..., 1.12, 1.11, 1.09])

- looping throught months Jan - Dec
- substituting 1 for Jan with the average exchange rate over that period and so on
    - using i-1 as index starts from 0

In [119]:
exchange_rate.shape

(10000,)

In [120]:
loan_data_numeric.shape

(10000, 6)

In [121]:
exchange_rate = np.reshape(exchange_rate, (10000,1))

In [123]:
loan_data_numeric = np.hstack((loan_data_numeric, exchange_rate))

In [124]:
header_numeric = np.concatenate((header_numeric, np.array(["exchange_rate"])))
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt', 'exchange_rate'],
      dtype='<U19')

#### From USD to EUR

In [125]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt', 'exchange_rate'],
      dtype='<U19')

- loan amount, funded amount, installment, total payment == $

In [127]:
columns_dollar = np.array([1,2,4,5])

In [130]:
loan_data_numeric[:,6]

array([1.1 , 1.11, 1.13, ..., 1.12, 1.11, 1.09])

In [131]:
for i in columns_dollar:
    loan_data_numeric = np.hstack((loan_data_numeric, np.reshape(loan_data_numeric[:,i] / loan_data_numeric[:,6], (10000,1))))

- in each iteration, we're creating a new array and are horizontally stacking it to the end of the data set
- we're setting loan_data_numeric to the horizontal stacking function 
    - first input is the entire data (we want to attach the new columns to the existing data set)
    - second input will be referencing a column depending on the iterator variable and divided by the exchange rate
        - as the EUR->USD show how much a single euro is worth in dollars, we need to divide by the exchange rate rather than multiply to get the corresponding numbers in euros

    - need to reshape the vector into a matrix

In [133]:
loan_data_numeric.shape

(10000, 11)

In [134]:
loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  , ...,    31933.3 ,     1081.04,     8624.69],
       [57693261.  ,    30000.  ,    30000.  , ...,    27132.46,      848.86,     4232.39],
       [59432726.  ,    15000.  ,    15000.  , ...,    13326.3 ,      439.64,     1750.04],
       ...,
       [50415990.  ,    10000.  ,    10000.  , ...,     8910.3 ,     1223.36,     1947.47],
       [46154151.  ,    35000.  ,    10000.  , ...,     8997.4 ,      318.78,     2878.63],
       [66055249.  ,    10000.  ,    10000.  , ...,     9145.8 ,      283.49,      276.11]])

as we've added new columns to the dataset, we also need to update the header

#### Expanding the header

In [136]:
header_additional = np.array([column_name + '_EUR' for column_name in header_numeric[columns_dollar]])

- list comprehension creates an array by going through the headers for the dollar columns and attaching _EUR after each one

In [138]:
header_numeric = np.concatenate((header_numeric, header_additional))

In [139]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt', 'exchange_rate',
       'loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'], dtype='<U19')

In [140]:
header_numeric[columns_dollar] = np.array([column_name + '_USD' for column_name in header_numeric[columns_dollar]])

In [141]:
header_numeric

array(['id', 'loan_amnt_USD', 'funded_amnt_USD', 'int_rate', 'installment_USD', 'total_pymnt_USD',
       'exchange_rate', 'loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'],
      dtype='<U19')

- rearrange columns so that each EUR columns follows its corresponding USD column

- create a list which contains all the indices that will arrange the data set and the header in our desired order

In [142]:
columns_index_order = [0,1,7,2,8,3,4,9,5,10,6]

In [144]:
header_numeric = header_numeric[columns_index_order]

In [145]:
loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  , ...,    31933.3 ,     1081.04,     8624.69],
       [57693261.  ,    30000.  ,    30000.  , ...,    27132.46,      848.86,     4232.39],
       [59432726.  ,    15000.  ,    15000.  , ...,    13326.3 ,      439.64,     1750.04],
       ...,
       [50415990.  ,    10000.  ,    10000.  , ...,     8910.3 ,     1223.36,     1947.47],
       [46154151.  ,    35000.  ,    10000.  , ...,     8997.4 ,      318.78,     2878.63],
       [66055249.  ,    10000.  ,    10000.  , ...,     9145.8 ,      283.49,      276.11]])

In [146]:
loan_data_numeric = loan_data_numeric[:,columns_index_order]

So far:
- filled out any missing values appropriately
- added exchange rates for each applicant (account)
- created EUR versions of the 4 monetary values

### Interest Rate

In [147]:
header_numeric

array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
       'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR', 'exchange_rate'],
      dtype='<U19')

In [148]:
loan_data_numeric[:,5]

array([13.33, 28.99, 28.99, ..., 28.99, 16.55, 28.99])

- change interest rates from %s to values from 0-1
- makes calculations e.g. estimating interest payments simpler

In [150]:
loan_data_numeric[:,5] = loan_data_numeric[:,5] / 100

In [151]:
loan_data_numeric[:,5]

array([0.13, 0.29, 0.29, ..., 0.29, 0.17, 0.29])

### Checkpoint 2: Numeric

In [152]:
checkpoint_numeric = checkpoint("Checkpoint-Numeric", header_numeric, loan_data_numeric)

In [153]:
checkpoint_numeric['header'], checkpoint_numeric['data']

(array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
        'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR', 'exchange_rate'],
       dtype='<U19'),
 array([[48010226.  ,    35000.  ,    31933.3 , ...,     9452.96,     8624.69,        1.1 ],
        [57693261.  ,    30000.  ,    27132.46, ...,     4679.7 ,     4232.39,        1.11],
        [59432726.  ,    15000.  ,    13326.3 , ...,     1969.83,     1750.04,        1.13],
        ...,
        [50415990.  ,    10000.  ,     8910.3 , ...,     2185.64,     1947.47,        1.12],
        [46154151.  ,    35000.  ,    31490.9 , ...,     3199.4 ,     2878.63,        1.11],
        [66055249.  ,    10000.  ,     9145.8 , ...,      301.9 ,      276.11,        1.09]]))

## Creating the "Complete" Dataset

In [154]:
checkpoint_strings["data"].shape

(10000, 6)

In [155]:
checkpoint_numeric["data"].shape

(10000, 11)

- use hstack on both arrays
- start with checkpoint_numeric as it contains all ID's in first column

In [161]:
loan_data = np.hstack((checkpoint_numeric["data"], checkpoint_strings["data"]))

In [162]:
loan_data

array([[48010226.  ,    35000.  ,    31933.3 , ...,       13.  ,        1.  ,        1.  ],
       [57693261.  ,    30000.  ,    27132.46, ...,        5.  ,        1.  ,        4.  ],
       [59432726.  ,    15000.  ,    13326.3 , ...,       10.  ,        1.  ,        4.  ],
       ...,
       [50415990.  ,    10000.  ,     8910.3 , ...,        5.  ,        1.  ,        1.  ],
       [46154151.  ,    35000.  ,    31490.9 , ...,       17.  ,        1.  ,        3.  ],
       [66055249.  ,    10000.  ,     9145.8 , ...,        4.  ,        0.  ,        3.  ]])

In [160]:
np.isnan(loan_data).sum()

0

In [164]:
header_full = np.concatenate((checkpoint_numeric["header"], checkpoint_strings["header"]))

## Sorting the New Dataset

- rearrange entire dataset according to values in the first column

In [168]:
loan_data = loan_data[np.argsort(loan_data[:,0])]

In [169]:
loan_data

array([[  373332.  ,     9950.  ,     9038.08, ...,       21.  ,        0.  ,        1.  ],
       [  575239.  ,    12000.  ,    10900.2 , ...,       25.  ,        1.  ,        2.  ],
       [  707689.  ,    10000.  ,     8924.3 , ...,       13.  ,        1.  ,        0.  ],
       ...,
       [68614880.  ,     5600.  ,     5121.65, ...,        8.  ,        1.  ,        1.  ],
       [68615915.  ,     4000.  ,     3658.32, ...,       10.  ,        1.  ,        2.  ],
       [68616519.  ,    21600.  ,    19754.93, ...,        3.  ,        0.  ,        2.  ]])

In [170]:
np.argsort(loan_data[:,0])

array([   0,    1,    2, ..., 9997, 9998, 9999], dtype=int64)

## Storing the New Dataset

- stack header on top of data set using vstack
    - everything will be converted to strings as stack requires a unified datatype across all rows and columns
        - function selects smallest datatype which can hold any of the elements within the array

In [172]:
loan_data = np.vstack((header_full, loan_data))

In [173]:
loan_data

array([['id', 'loan_amnt_USD', 'loan_amnt_EUR', ..., 'sub_grade', 'verification_status',
        'state_address'],
       ['373332.0', '9950.0', '9038.082814338286', ..., '21.0', '0.0', '1.0'],
       ['575239.0', '12000.0', '10900.20037910145', ..., '25.0', '1.0', '2.0'],
       ...,
       ['68614880.0', '5600.0', '5121.647851612413', ..., '8.0', '1.0', '1.0'],
       ['68615915.0', '4000.0', '3658.319894008867', ..., '10.0', '1.0', '2.0'],
       ['68616519.0', '21600.0', '19754.927427647883', ..., '3.0', '0.0', '2.0']], dtype='<U32')

In [174]:
np.savetxt("loan-data-preprocessed.csv", loan_data, fmt="%s", delimiter=",")